In [1]:
import tensorflow as tf
import pandas as pd
import math
import numpy as np

2025-01-19 18:49:15.467847: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-19 18:49:15.553990: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737308955.589170   51521 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737308955.599111   51521 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-19 18:49:15.611665: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# df_true = pd.read_csv("dataset/True.csv")
# df_fake = pd.read_csv("dataset/Fake.csv")

In [3]:
df_train = pd.read_csv("dataset/train.csv", sep = ";")
df_test = pd.read_csv("dataset/test.csv", sep = ";")
df_eval = pd.read_csv("dataset/evaluation.csv", sep = ";")

In [4]:
# all_titles = pd.concat([df_true, df_fake]).dropna()["title"].tolist()
# titles_2 = pd.concat([df_train, df_eval, df_test]).dropna()["title"].tolist()
all_titles = pd.concat([df_train, df_eval, df_test]).dropna()["title"].tolist()

In [5]:
# all_titles.extend(titles_2)

In [6]:
lengths = [len(x) for x in all_titles]
mean_len = math.floor(np.mean(lengths))
print(f"min  {min(lengths)}")
print(f"max  {max(lengths)}")
print(f"mean {np.mean(lengths)}")

min  2
max  443
mean 75.95816394412005


In [7]:
vocab = sorted(set("".join(all_titles)))
print(f"vocab: {vocab}")

vocab: ['\n', '\r', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '\xa0', '¡', '£', '\xad', '¿', 'É', 'ß', 'à', 'á', 'â', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'ñ', 'ö', 'ü', 'ć', 'Ž', 'ž', '\u200b', '–', '—', '‘', '’', '“', '”', '…', '\u202a', '\u202c', '′', '″', '€', '™', '\ufeff']


In [8]:
filter_vocab = set(sorted([
    ' ', '!', '"', '#', '$',
    '%','&', "'", '(', ')',
    '*', '+', ',', '-','.',
    '/', '0', '1', '2', '3',
    '4', '5', '6', '7', '8',
    '9', ':', ';', '=', '?',
    '@', 'A', 'B', 'C', 'D',
    'E', 'F', 'G', 'H', 'I',
    'J', 'K', 'L', 'M', 'N',
    'O', 'P', 'Q', 'R', 'S',
    'T', 'U', 'V', 'W', 'X',
    'Y', 'Z', '[', ']', '_',
    'a', 'b', 'c', 'd', 'e',
    'f', 'g', 'h', 'i', 'j',
    'k', 'l', 'm', 'n', 'o',
    'p', 'q', 'r', 's', 't',
    'u', 'v', 'w', 'x', 'y',
    'z', '{', '}', '“', '”',
    "<pad>"
]))

In [9]:
VOCAB_SIZE: int = len(filter_vocab)
SEQ_SIZE: int = 100

In [10]:
def filter_chars(text: str) -> str:
    global filter_vocab
    return "".join([c if c in filter_vocab else "" for c in text])

In [11]:
filtered_titles = [filter_chars(title) for title in all_titles]
filtered_titles

['Palestinians switch off Christmas lights in Bethlehem in anti-Trump protest',
 "China says Trump call with Taiwan president won't change island's status",
 ' FAIL! The Trump Organizations Credit Score Will Make You Laugh',
 "Zimbabwe military chief's China trip was normal visit, Beijing says",
 'THE MOST UNCOURAGEOUS PRESIDENT EVER Receives A Courage AwardProceeds To Whine About Current President',
 'Suspected Boko Haram suicide bombers kill at least 13 in Nigeria: officials',
 'WATCH: John Oliver Presents GOP Debates As Clowntown F*ck-the-World Sh*tshow 2016',
 'Senate Democrats ask Trump attorney general pick to recuse himself from Russia probes',
 ' Trump HUMILIATES Republicans In Latest Hissy Fit After Siding With Democrats On Debt Ceiling',
 'MACYS GETS THE BOOT FROM LOYAL CUSTOMERS AFTER FIRING TRUMP',
 'A North Korea nuclear test over the Pacific? Logical, terrifying',
 'Czech police ask parliament to allow prosecution of prospective PM Babis',
 'ONE DEMOCRAT WHO REFUSES To Ca

In [12]:
def get_encoder(vocab: set) -> dict:
    one_hot_encoder = {}
    for key, val in zip(vocab, np.eye(len(vocab), dtype=np.float32)):
        one_hot_encoder[key] = list(val)
    return one_hot_encoder
one_hot_encoder = get_encoder(filter_vocab)

In [13]:
import pprint
pprint.pprint(one_hot_encoder)

{' ': [np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(1.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float32(0.0),
       np.float3

In [14]:
r = set([np.argmax(one_hot_encoder[c]) for c in filter_vocab])

In [15]:
len(r)

91

In [16]:
def split_encode(text: str) -> list[list[int]]:
    global one_hot_encoder
    global SEQ_SIZE
    encoded = [one_hot_encoder[x] for x in [*text][:SEQ_SIZE]]
    padded = encoded + [one_hot_encoder["<pad>"]] * (SEQ_SIZE - len(encoded))
    return padded

In [17]:
data = np.array([split_encode(text) for text in filtered_titles])

In [18]:
data.shape

(40587, 100, 91)

In [19]:
BATCH_SIZE = 8

In [20]:
X = data[:, :-1, :] 
y = data[:, 1:, :]  

In [21]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(512).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

W0000 00:00:1737308966.574131   51521 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [22]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(SEQ_SIZE - 1, VOCAB_SIZE)),
    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.Dense(VOCAB_SIZE, activation='softmax')
])

In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [24]:
history = model.fit(dataset, epochs=20)

Epoch 1/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 155s 30ms/step - loss: 1.9784
Epoch 2/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 154s 30ms/step - loss: 1.2852
Epoch 3/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 152s 30ms/step - loss: 1.1163
Epoch 4/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 154s 30ms/step - loss: 1.0432
Epoch 5/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 156s 31ms/step - loss: 1.0003
Epoch 6/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 157s 31ms/step - loss: 0.9723
Epoch 7/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 155s 31ms/step - loss: 0.9514
Epoch 8/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 154s 30ms/step - loss: 0.9357
Epoch 9/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 158s 31ms/step - loss: 0.9236
Epoch 10/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 156s 31ms/step - loss: 0.9127
Epoch 11/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 156s 31ms/step - loss: 0.9039
Epoch 12/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 148s 29ms/step - loss: 0.8962
Epoch 13/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 147s 29ms/step - loss: 0.8894
Epoch 14/20
5074/5074 ━━━━━━━━━━━━━━━━━━━━ 162s 32ms/step - 

In [29]:
def encode(src: str) -> np.ndarray:
    dst = []
    for letter in [*src]:
        dst.append(one_hot_encoder[letter])
    return np.array(dst)

In [38]:
def decode(src: np.ndarray) -> str:
    dst = ""
    for coded in src:
        for key, val in one_hot_encoder.items():
            if (coded == val).all():
                dst += key
                break
    return dst

In [41]:
input_seq = encode("Breaking news: ")

In [2]:
def generate_sequence(model, start_sequence, seq_length, decode):
    current_input = np.copy(start_sequence)  
    generated_sequence = decode(current_input)  
    for _ in range(seq_length):
        predictions = model.predict(current_input[np.newaxis, ...]) 
        
        next_char_idx = np.argmax(predictions[0, -1, :])
        
        next_char_onehot = np.zeros_like(current_input[0])
        next_char_onehot[next_char_idx] = 1

        current_input = np.roll(current_input, shift=-1, axis=0)
        current_input[-1] = next_char_onehot  
        
        generated_sequence += decode(next_char_onehot[np.newaxis, ...])
    
    return generated_sequence

In [43]:

seq_length = 100

generated_text = generate_sequence(model, input_seq, seq_length, decode)
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━

In [45]:
model.save("generator.keras")

In [46]:
history

In [1]:
import matplotlib.pyplot as plt

history_dict = history.history

loss = history_dict['loss']  
epochs = range(1, len(loss) + 1)  

val_loss = history_dict.get('val_loss', None)
accuracy = history_dict.get('accuracy', None)
val_accuracy = history_dict.get('val_accuracy', None)

plt.figure(figsize=(12, 5))

plt.plot(epochs, loss, 'bo-', label='Training Loss') 
if val_loss:
    plt.plot(epochs, val_loss, 'ro-', label='Validation Loss') 
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

NameError: name 'history' is not defined